<a href="https://colab.research.google.com/github/ravi-gopalan/ipl_analysis/blob/master/ipl_yaml_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get data in YAML files and process it
- Mount Google Drive
- Get Kaggle data

In [1]:
!ls -la

total 16
drwxr-xr-x 1 root root 4096 Aug 27 16:17 .
drwxr-xr-x 1 root root 4096 Sep 16 15:10 ..
drwxr-xr-x 1 root root 4096 Sep  9 16:05 .config
drwxr-xr-x 1 root root 4096 Aug 27 16:17 sample_data


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
cd /content

/content


In [0]:
!mkdir ipl

In [5]:
cd /content/ipl

/content/ipl


In [6]:
cd /root

/root


In [0]:
!mkdir .kaggle

In [0]:
!cp '/content/gdrive/My Drive/Colab_stuff/kaggle.json' /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [9]:
cd /content/ipl

/content/ipl


In [10]:
!kaggle datasets download -d ravijee/ipl-2008-to-2019 -p /content/ipl

100% 4.07M/4.07M [00:00<00:00, 28.5MB/s]



In [11]:
!ls -la

total 4180
drwxr-xr-x 2 root root    4096 Sep 16 15:57 .
drwxr-xr-x 1 root root    4096 Sep 16 15:56 ..
-rw-r--r-- 1 root root 4268495 Sep 16 15:57 ipl-2008-to-2019.zip


In [12]:
pwd

'/content/ipl'

In [0]:
!unzip -q /content/ipl/ipl-2008-to-2019.zip

In [14]:
!pip install ruamel.yaml

     |████████████████████████████████| 133kB 5.0MB/s 
     |████████████████████████████████| 552kB 16.4MB/s 


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from ruamel.yaml import YAML
import json
import datetime as dt

In [0]:
base_dir = Path('/content/ipl')

In [17]:
!ls -la

total 60900
drwxr-xr-x 2 root root     4096 Sep 16 15:57 .
drwxr-xr-x 1 root root     4096 Sep 16 15:56 ..
---------- 1 root root 55848188 Jun 18 00:41 ball_by_ball_details.csv
-rw-r--r-- 1 root root  4268495 Sep 16 15:57 ipl-2008-to-2019.zip
---------- 1 root root   166458 Jun 18 00:41 ipl_summary.csv
---------- 1 root root  2060580 Jun 18 00:41 ipl.zip


In [0]:
!unzip -q /content/ipl/ipl.zip

In [0]:
yaml_file_list = list(Path(base_dir).glob('*.yaml'))

In [21]:
len(yaml_file_list)

756

In [0]:
def myconverter(o):
    if isinstance(o, dt.date):
        return o.__str__()

In [0]:
def create_data_frame_from_innings_dict(datafile_json, inning_number, innings_dict):
  '''
  This function takes a json data file, the inning_number and the innings_dict and converts it into a dataframe
  INPUT - datafile_json, inning_number, innings_dict 
  OUTPUT - DataFrame
  '''
  df_innings=pd.DataFrame({'col':datafile_json['innings'][inning_number][innings_dict]['deliveries']})
  df_T = df_innings['col'].apply(pd.Series).T
  df_T['main'] = df_T[df_T.columns].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
  df_next = df_T.filter(['main'])
  df_next['main'] = df_next['main'].apply(lambda x : eval(x)) 
  df_next_2 = df_next['main'].apply(pd.Series)
  df_next_try = df_next_2.filter(['batsman','bowler','non_striker','runs','wicket','extras'])

  df_runs = df_next_try[['runs']]
  df_run_details = df_runs['runs'].apply(pd.Series)
  
  if 'wicket' in df_next_try.columns:
    df_wickets = df_next_try[['wicket']]
    df_wicket_details = df_wickets['wicket'].apply(pd.Series)
  else:
    df_wicket_details = pd.DataFrame(columns=['fielders', 'kind', 'player_out'])   
         
  if 'extras' in df_next_try.columns:
    df_extras = df_next_try[['extras']]
    df_extras_details = df_extras['extras'].apply(pd.Series)
  else:
    df_extras_details = pd.DataFrame(columns=['0', 'wides', 'legbyes'])
    
  drop_column_list = ['runs']

  if 'wicket' in df_next_try.columns:
    drop_column_list.append('wicket')

  if 'extras' in df_next_try.columns:
    drop_column_list.append('extras')
  
  df = df_next_try.join(df_run_details,rsuffix='_runs').join(df_wicket_details,rsuffix='_wickets').join(df_extras_details,rsuffix='_extras').drop(columns=drop_column_list)
  df['batting_team'] = datafile_json['innings'][inning_number][innings_dict]['team']
  
  return df


In [0]:
def create_data_frame_from_info_dict(datafile_json):
  '''
  This function creates a match_summary dataframe given an input json datafile
  '''
  info = pd.DataFrame({'info':datafile_json['info']}).T
  outcome = info['outcome'].apply(pd.Series)
  toss = info['toss'].apply(pd.Series)
  match_summary = info.join(outcome,rsuffix='_outcome').join(toss, rsuffix='_toss').drop(columns=['outcome','toss'])
  return match_summary

In [24]:
len(yaml_file_list)

756

In [25]:
counter = 0
info_list = []
ball_by_ball_details = []
for each_file in range(len(yaml_file_list)):
  in_file = yaml_file_list[each_file]
  out_file = 'output.json'


  tyaml = YAML(typ='safe')
  with open(in_file) as fpi:
      data_yaml = tyaml.load(fpi)
  with open(out_file, 'w') as fpo:
      json.dump(data_yaml, fpo, default=myconverter)
    
  with open(out_file, 'r') as f:
      json_file = json.load(f)
      
      
  if len(json_file['innings']) == 2:
    d1 = create_data_frame_from_innings_dict(datafile_json=json_file, inning_number=0, innings_dict='1st innings')
    d2 = create_data_frame_from_innings_dict(datafile_json=json_file, inning_number=1, innings_dict='2nd innings')
    df_match = pd.concat([d1, d2], keys=['inning_1', 'inning_2']).reset_index()
  elif len(json_file['innings']) == 1:
    d1 = create_data_frame_from_innings_dict(datafile_json=json_file, inning_number=0, innings_dict='1st innings')
    d2 = pd.DataFrame()
    df_match = pd.concat([d1, d2], keys=['inning_1', 'inning_2']).reset_index()
  elif len(json_file['innings']) == 4:
    d1 = create_data_frame_from_innings_dict(datafile_json=json_file, inning_number=0, innings_dict='1st innings')
    d2 = create_data_frame_from_innings_dict(datafile_json=json_file, inning_number=1, innings_dict='2nd innings')    
    df_match = pd.concat([d1, d2], keys=['inning_1', 'inning_2']).reset_index()
  else:
    df_match = pd.DataFrame()
  
  df_match['match_id'] = yaml_file_list[each_file].stem
  match_info_summary = create_data_frame_from_info_dict(datafile_json=json_file)
  
  match_info_summary['match_id'] = yaml_file_list[each_file].stem

  
  ball_by_ball_details.append(df_match)
  info_list.append(match_info_summary)
  counter += 1
  if counter % 50 == 0:
    print(counter,' files completed')
#  print(counter,':' ,yaml_file_list[each_file].stem)
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



50  files completed
100  files completed
150  files completed
200  files completed
250  files completed
300  files completed
350  files completed
400  files completed
450  files completed
500  files completed
550  files completed


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



600  files completed
650  files completed
700  files completed
750  files completed


In [26]:
df_match_summary = pd.concat(info_list)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [0]:
df_match_summary = df_match_summary.set_index('match_id')

In [28]:
df_match_summary.head()

,by,city,competition,dates,decision,eliminator,gender,match_type,method,neutral_venue,overs,player_of_match,result,teams,umpires,venue,winner,winner_toss
match_id,,,,,,,,,,,,,,,,,,
336036,{'wickets': 5},Jaipur,IPL,[2008-05-26],field,NaN,male,T20,NaN,NaN,20,[Sohail Tanvir],NaN,"[Rajasthan Royals, Mumbai Indians]","[BF Bowden, K Hariharan]",Sawai Mansingh Stadium,Rajasthan Royals,Rajasthan Royals
548353,{'runs': 7},Kolkata,IPL,[2012-05-05],bat,NaN,male,T20,NaN,NaN,20,[SP Narine],NaN,"[Kolkata Knight Riders, Pune Warriors]","[BF Bowden, SK Tarapore]",Eden Gardens,Kolkata Knight Riders,Kolkata Knight Riders
829779,{'runs': 24},Chennai,IPL,[2015-05-04],bat,NaN,male,T20,NaN,NaN,20,[SK Raina],NaN,"[Chennai Super Kings, Royal Challengers Bangal...","[C Shamshuddin, K Srinath]","MA Chidambaram Stadium, Chepauk",Chennai Super Kings,Chennai Super Kings
829785,{'runs': 138},Bangalore,IPL,[2015-05-06],field,NaN,male,T20,NaN,NaN,20,[CH Gayle],NaN,"[Royal Challengers Bangalore, Kings XI Punjab]","[RK Illingworth, VA Kulkarni]",M Chinnaswamy Stadium,Royal Challengers Bangalore,Kings XI Punjab
548308,{'runs': 28},Mumbai,IPL,[2012-04-06],field,NaN,male,T20,NaN,NaN,20,[SPD Smith],NaN,"[Mumbai Indians, Pune Warriors]","[AK Chaudhary, SJA Taufel]",Wankhede Stadium,Pune Warriors,Mumbai Indians


In [29]:
df_details = pd.concat(ball_by_ball_details)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [30]:
df_details.head()

,0,0_extras,0_wickets,batsman,batsman_runs,batting_team,bowler,byes,extras_runs,fielders,kind,legbyes,level_0,level_1,match_id,noballs,non_boundary,non_striker,player_out,total,wides,0,extras,penalty
0,NaN,NaN,NaN,ST Jayasuriya,0.0,Mumbai Indians,Sohail Tanvir,NaN,0.0,NaN,NaN,NaN,inning_1,0.1,336036,NaN,NaN,SR Tendulkar,NaN,0.0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,ST Jayasuriya,0.0,Mumbai Indians,Sohail Tanvir,NaN,0.0,NaN,NaN,NaN,inning_1,0.2,336036,NaN,NaN,SR Tendulkar,NaN,0.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,ST Jayasuriya,0.0,Mumbai Indians,Sohail Tanvir,NaN,0.0,NaN,NaN,NaN,inning_1,0.3,336036,NaN,NaN,SR Tendulkar,NaN,0.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,ST Jayasuriya,0.0,Mumbai Indians,Sohail Tanvir,NaN,0.0,NaN,NaN,NaN,inning_1,0.4,336036,NaN,NaN,SR Tendulkar,NaN,0.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,ST Jayasuriya,0.0,Mumbai Indians,Sohail Tanvir,NaN,1.0,NaN,NaN,NaN,inning_1,0.5,336036,NaN,NaN,SR Tendulkar,NaN,1.0,1,NaN,NaN,NaN


In [0]:
df_details['wides'] = df_details['wides'].astype('float')
df_details['noballs'] = df_details['noballs'].astype('float')
df_details['legbyes'] = df_details['legbyes'].astype('float')


In [32]:
df_details.iloc[:,:].isna().mean()

0               1.000000
0_extras        1.000000
0_wickets       1.000000
batsman         0.000050
batsman_runs    0.000050
batting_team    0.000000
bowler          0.000050
byes            0.997318
extras_runs     0.003632
fielders        0.964582
kind            0.950673
legbyes         0.983610
level_0         0.000000
level_1         0.000000
match_id        0.000000
noballs         0.996021
non_boundary    0.999922
non_striker     0.000050
player_out      0.950673
total           0.000050
wides           0.969796
0               1.000000
extras          0.996418
penalty         0.999989
dtype: float64

In [0]:
df_details.drop(columns=['0','0_extras','0_wickets'], inplace=True, axis=1)
df_details.drop(columns=[0], inplace=True, axis=1)

In [34]:
df_details.iloc[:,:].isna().mean()

batsman         0.000050
batsman_runs    0.000050
batting_team    0.000000
bowler          0.000050
byes            0.997318
extras_runs     0.003632
fielders        0.964582
kind            0.950673
legbyes         0.983610
level_0         0.000000
level_1         0.000000
match_id        0.000000
noballs         0.996021
non_boundary    0.999922
non_striker     0.000050
player_out      0.950673
total           0.000050
wides           0.969796
extras          0.996418
penalty         0.999989
dtype: float64

In [35]:
df_details.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178949 entries, 0 to 226
Data columns (total 20 columns):
batsman         178940 non-null object
batsman_runs    178940 non-null float64
batting_team    178949 non-null object
bowler          178940 non-null object
byes            480 non-null float64
extras_runs     178299 non-null float64
fielders        6338 non-null object
kind            8827 non-null object
legbyes         2933 non-null float64
level_0         178949 non-null object
level_1         178949 non-null object
match_id        178949 non-null object
noballs         712 non-null float64
non_boundary    14 non-null float64
non_striker     178940 non-null object
player_out      8827 non-null object
total           178940 non-null float64
wides           5405 non-null float64
extras          641 non-null float64
penalty         2 non-null float64
dtypes: float64(10), object(10)
memory usage: 28.7+ MB


In [0]:
yaml_match_id_list = [i.stem for i in yaml_file_list]

In [0]:
unmatched_yaml = list(set(yaml_match_id_list) - set(df_details.match_id.unique()))

In [0]:
unmatched_yaml = [i+'.yaml' for i in unmatched_yaml]

In [39]:
unmatched_yaml

[]

In [40]:
len(json_file['innings'])

2

In [0]:
df_outcome = df_match_summary[['by']]
df_outcome_details = df_outcome['by'].apply(pd.Series).drop(columns=[0])

In [0]:
df_ipl_summary = df_match_summary.join(df_outcome_details)

In [0]:
df_ipl_summary.drop(columns=['by'],axis=1,inplace=True)

In [44]:
df_ipl_summary=df_ipl_summary.rename(index=str, columns={"runs": "outcome_by_runs", "wickets": "outcome_by_wickets", "overs": "#_of_overs"})
df_ipl_summary['match_date'] = df_ipl_summary['dates'].str[0]
df_ipl_summary['match_date'] = pd.to_datetime(df_ipl_summary['match_date'])

df_ipl_summary['season'] = df_ipl_summary['match_date'].dt.year
df_ipl_summary.drop(columns=['dates'],axis=1,inplace=True)
df_ipl_summary.head()

,city,competition,decision,eliminator,gender,match_type,method,neutral_venue,#_of_overs,player_of_match,result,teams,umpires,venue,winner,winner_toss,outcome_by_runs,outcome_by_wickets,match_date,season
match_id,,,,,,,,,,,,,,,,,,,,
336036,Jaipur,IPL,field,NaN,male,T20,NaN,NaN,20,[Sohail Tanvir],NaN,"[Rajasthan Royals, Mumbai Indians]","[BF Bowden, K Hariharan]",Sawai Mansingh Stadium,Rajasthan Royals,Rajasthan Royals,NaN,5.0,2008-05-26,2008
548353,Kolkata,IPL,bat,NaN,male,T20,NaN,NaN,20,[SP Narine],NaN,"[Kolkata Knight Riders, Pune Warriors]","[BF Bowden, SK Tarapore]",Eden Gardens,Kolkata Knight Riders,Kolkata Knight Riders,7.0,NaN,2012-05-05,2012
829779,Chennai,IPL,bat,NaN,male,T20,NaN,NaN,20,[SK Raina],NaN,"[Chennai Super Kings, Royal Challengers Bangal...","[C Shamshuddin, K Srinath]","MA Chidambaram Stadium, Chepauk",Chennai Super Kings,Chennai Super Kings,24.0,NaN,2015-05-04,2015
829785,Bangalore,IPL,field,NaN,male,T20,NaN,NaN,20,[CH Gayle],NaN,"[Royal Challengers Bangalore, Kings XI Punjab]","[RK Illingworth, VA Kulkarni]",M Chinnaswamy Stadium,Royal Challengers Bangalore,Kings XI Punjab,138.0,NaN,2015-05-06,2015
548308,Mumbai,IPL,field,NaN,male,T20,NaN,NaN,20,[SPD Smith],NaN,"[Mumbai Indians, Pune Warriors]","[AK Chaudhary, SJA Taufel]",Wankhede Stadium,Pune Warriors,Mumbai Indians,28.0,NaN,2012-04-06,2012


In [45]:
df_ipl_summary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 756 entries, 336036 to 1178408
Data columns (total 20 columns):
city                  743 non-null object
competition           756 non-null object
decision              756 non-null object
eliminator            9 non-null object
gender                756 non-null object
match_type            756 non-null object
method                19 non-null object
neutral_venue         77 non-null object
#_of_overs            756 non-null object
player_of_match       752 non-null object
result                13 non-null object
teams                 756 non-null object
umpires               755 non-null object
venue                 756 non-null object
winner                756 non-null object
winner_toss           743 non-null object
outcome_by_runs       337 non-null float64
outcome_by_wickets    406 non-null float64
match_date            756 non-null datetime64[ns]
season                756 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(1)

In [46]:
df_ipl_summary.head()

,city,competition,decision,eliminator,gender,match_type,method,neutral_venue,#_of_overs,player_of_match,result,teams,umpires,venue,winner,winner_toss,outcome_by_runs,outcome_by_wickets,match_date,season
match_id,,,,,,,,,,,,,,,,,,,,
336036,Jaipur,IPL,field,NaN,male,T20,NaN,NaN,20,[Sohail Tanvir],NaN,"[Rajasthan Royals, Mumbai Indians]","[BF Bowden, K Hariharan]",Sawai Mansingh Stadium,Rajasthan Royals,Rajasthan Royals,NaN,5.0,2008-05-26,2008
548353,Kolkata,IPL,bat,NaN,male,T20,NaN,NaN,20,[SP Narine],NaN,"[Kolkata Knight Riders, Pune Warriors]","[BF Bowden, SK Tarapore]",Eden Gardens,Kolkata Knight Riders,Kolkata Knight Riders,7.0,NaN,2012-05-05,2012
829779,Chennai,IPL,bat,NaN,male,T20,NaN,NaN,20,[SK Raina],NaN,"[Chennai Super Kings, Royal Challengers Bangal...","[C Shamshuddin, K Srinath]","MA Chidambaram Stadium, Chepauk",Chennai Super Kings,Chennai Super Kings,24.0,NaN,2015-05-04,2015
829785,Bangalore,IPL,field,NaN,male,T20,NaN,NaN,20,[CH Gayle],NaN,"[Royal Challengers Bangalore, Kings XI Punjab]","[RK Illingworth, VA Kulkarni]",M Chinnaswamy Stadium,Royal Challengers Bangalore,Kings XI Punjab,138.0,NaN,2015-05-06,2015
548308,Mumbai,IPL,field,NaN,male,T20,NaN,NaN,20,[SPD Smith],NaN,"[Mumbai Indians, Pune Warriors]","[AK Chaudhary, SJA Taufel]",Wankhede Stadium,Pune Warriors,Mumbai Indians,28.0,NaN,2012-04-06,2012


In [47]:
df_details=df_details.rename(index=str, columns={"level_0": "inning", "level_1": "delivery"})
df_details.head()

,batsman,batsman_runs,batting_team,bowler,byes,extras_runs,fielders,kind,legbyes,inning,delivery,match_id,noballs,non_boundary,non_striker,player_out,total,wides,extras,penalty
0,ST Jayasuriya,0.0,Mumbai Indians,Sohail Tanvir,NaN,0.0,NaN,NaN,NaN,inning_1,0.1,336036,NaN,NaN,SR Tendulkar,NaN,0.0,NaN,NaN,NaN
1,ST Jayasuriya,0.0,Mumbai Indians,Sohail Tanvir,NaN,0.0,NaN,NaN,NaN,inning_1,0.2,336036,NaN,NaN,SR Tendulkar,NaN,0.0,NaN,NaN,NaN
2,ST Jayasuriya,0.0,Mumbai Indians,Sohail Tanvir,NaN,0.0,NaN,NaN,NaN,inning_1,0.3,336036,NaN,NaN,SR Tendulkar,NaN,0.0,NaN,NaN,NaN
3,ST Jayasuriya,0.0,Mumbai Indians,Sohail Tanvir,NaN,0.0,NaN,NaN,NaN,inning_1,0.4,336036,NaN,NaN,SR Tendulkar,NaN,0.0,NaN,NaN,NaN
4,ST Jayasuriya,0.0,Mumbai Indians,Sohail Tanvir,NaN,1.0,NaN,NaN,NaN,inning_1,0.5,336036,NaN,NaN,SR Tendulkar,NaN,1.0,1.0,NaN,NaN


In [0]:
df_details.drop(columns=['extras'],axis=1,inplace=True)

In [49]:
df_combined = df_details.join(df_ipl_summary,on='match_id',rsuffix='_ipl_summary')
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 178949 entries, 0 to 226
Data columns (total 39 columns):
batsman               178940 non-null object
batsman_runs          178940 non-null float64
batting_team          178949 non-null object
bowler                178940 non-null object
byes                  480 non-null float64
extras_runs           178299 non-null float64
fielders              6338 non-null object
kind                  8827 non-null object
legbyes               2933 non-null float64
inning                178949 non-null object
delivery              178949 non-null object
match_id              178949 non-null object
noballs               712 non-null float64
non_boundary          14 non-null float64
non_striker           178940 non-null object
player_out            8827 non-null object
total                 178940 non-null float64
wides                 5405 non-null float64
penalty               2 non-null float64
city                  175811 non-null object
competition  

In [50]:
df_combined.head()

,batsman,batsman_runs,batting_team,bowler,byes,extras_runs,fielders,kind,legbyes,inning,delivery,match_id,noballs,non_boundary,non_striker,player_out,total,wides,penalty,city,competition,decision,eliminator,gender,match_type,method,neutral_venue,#_of_overs,player_of_match,result,teams,umpires,venue,winner,winner_toss,outcome_by_runs,outcome_by_wickets,match_date,season
0,ST Jayasuriya,0.0,Mumbai Indians,Sohail Tanvir,NaN,0.0,NaN,NaN,NaN,inning_1,0.1,336036,NaN,NaN,SR Tendulkar,NaN,0.0,NaN,NaN,Jaipur,IPL,field,NaN,male,T20,NaN,NaN,20,[Sohail Tanvir],NaN,"[Rajasthan Royals, Mumbai Indians]","[BF Bowden, K Hariharan]",Sawai Mansingh Stadium,Rajasthan Royals,Rajasthan Royals,NaN,5.0,2008-05-26,2008
1,ST Jayasuriya,0.0,Mumbai Indians,Sohail Tanvir,NaN,0.0,NaN,NaN,NaN,inning_1,0.2,336036,NaN,NaN,SR Tendulkar,NaN,0.0,NaN,NaN,Jaipur,IPL,field,NaN,male,T20,NaN,NaN,20,[Sohail Tanvir],NaN,"[Rajasthan Royals, Mumbai Indians]","[BF Bowden, K Hariharan]",Sawai Mansingh Stadium,Rajasthan Royals,Rajasthan Royals,NaN,5.0,2008-05-26,2008
2,ST Jayasuriya,0.0,Mumbai Indians,Sohail Tanvir,NaN,0.0,NaN,NaN,NaN,inning_1,0.3,336036,NaN,NaN,SR Tendulkar,NaN,0.0,NaN,NaN,Jaipur,IPL,field,NaN,male,T20,NaN,NaN,20,[Sohail Tanvir],NaN,"[Rajasthan Royals, Mumbai Indians]","[BF Bowden, K Hariharan]",Sawai Mansingh Stadium,Rajasthan Royals,Rajasthan Royals,NaN,5.0,2008-05-26,2008
3,ST Jayasuriya,0.0,Mumbai Indians,Sohail Tanvir,NaN,0.0,NaN,NaN,NaN,inning_1,0.4,336036,NaN,NaN,SR Tendulkar,NaN,0.0,NaN,NaN,Jaipur,IPL,field,NaN,male,T20,NaN,NaN,20,[Sohail Tanvir],NaN,"[Rajasthan Royals, Mumbai Indians]","[BF Bowden, K Hariharan]",Sawai Mansingh Stadium,Rajasthan Royals,Rajasthan Royals,NaN,5.0,2008-05-26,2008
4,ST Jayasuriya,0.0,Mumbai Indians,Sohail Tanvir,NaN,1.0,NaN,NaN,NaN,inning_1,0.5,336036,NaN,NaN,SR Tendulkar,NaN,1.0,1.0,NaN,Jaipur,IPL,field,NaN,male,T20,NaN,NaN,20,[Sohail Tanvir],NaN,"[Rajasthan Royals, Mumbai Indians]","[BF Bowden, K Hariharan]",Sawai Mansingh Stadium,Rajasthan Royals,Rajasthan Royals,NaN,5.0,2008-05-26,2008


In [0]:
overs = {'0.1': '1','0.2': '1', '0.3': '1', '0.4': '1', '0.5': '1', '0.6': '1', '0.7': '1', '0.8': '1', '0.9': '1',\
         '1.1': '2','1.2': '2', '1.3': '2', '1.4': '2', '1.5': '2', '1.6': '2', '1.7': '2', '1.8': '2', '1.9': '2',\
         '2.1': '3','2.2': '3', '2.3': '3', '2.4': '3', '2.5': '3', '2.6': '3', '2.7': '3', '2.8': '3', '2.9': '3',\
         '3.1': '4','3.2': '4', '3.3': '4', '3.4': '4', '3.5': '4', '3.6': '4', '3.7': '4', '3.8': '4', '3.9': '4',\
         '4.1': '5','4.2': '5', '4.3': '5', '4.4': '5', '4.5': '5', '4.6': '5', '4.7': '5', '4.8': '5', '4.9': '5',\
         '5.1': '6','5.2': '6', '5.3': '6', '5.4': '6', '5.5': '6', '5.6': '6', '5.7': '6', '5.8': '6', '5.9': '6',\
         '6.1': '7','6.2': '7', '6.3': '7', '6.4': '7', '6.5': '7', '6.6': '7', '6.7': '7', '6.8': '7', '6.9': '7',\
         '7.1': '8','7.2': '8', '7.3': '8', '7.4': '8', '7.5': '8', '7.6': '8', '7.7': '8', '7.8': '8', '7.9': '8',\
         '8.1': '9','8.2': '9', '8.3': '9', '8.4': '9', '8.5': '9', '8.6': '9', '8.7': '9', '8.8': '9', '8.9': '9',\
         '9.1': '10','9.2': '10', '9.3': '10', '9.4': '10', '9.5': '10', '9.6': '10', '9.7': '10', '9.8': '10', '9.9': '10',\
         '10.1': '11','10.2': '11', '10.3': '11', '10.4': '11', '10.5': '11', '10.6': '11', '10.7': '11', '10.8': '11', '10.9': '11',\
         '11.1': '12','11.2': '12', '11.3': '12', '11.4': '12', '11.5': '12', '11.6': '12', '11.7': '12', '11.8': '12', '11.9': '12',\
         '12.1': '13','12.2': '13', '12.3': '13', '12.4': '13', '12.5': '13', '12.6': '13', '12.7': '13', '12.8': '13', '12.9': '13',\
         '13.1': '14','13.2': '14', '13.3': '14', '13.4': '14', '13.5': '14', '13.6': '14', '13.7': '14', '13.8': '14', '13.9': '14',\
         '14.1': '15','14.2': '15', '14.3': '15', '14.4': '15', '14.5': '15', '14.6': '15', '14.7': '15', '14.8': '15', '14.9': '15',\
         '15.1': '16','15.2': '16', '15.3': '16', '15.4': '16', '15.5': '16', '15.6': '16', '15.7': '16', '15.8': '16', '15.9': '16',\
         '16.1': '17','16.2': '17', '16.3': '17', '16.4': '17', '16.5': '17', '16.6': '17', '16.7': '17', '16.8': '17', '16.9': '17',\
         '17.1': '18','17.2': '18', '17.3': '18', '17.4': '18', '17.5': '18', '17.6': '18', '17.7': '18', '17.8': '18', '17.9': '18',\
         '18.1': '19','18.2': '19', '18.3': '19', '18.4': '19', '18.5': '19', '18.6': '19', '18.7': '19', '18.8': '19', '18.9': '19',\
         '19.1': '20','19.2': '20', '19.3': '20', '19.4': '20', '19.5': '20', '19.6': '20', '19.7': '20', '19.8': '20', '19.9': '20',\
        }

In [0]:
df_combined['over'] = df_combined['delivery'].map(overs)

In [53]:
df_combined['delivery'].unique()

array(['0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '1.1', '1.2',
       '1.3', '1.4', '1.5', '1.6', '1.7', '2.1', '2.2', '2.3', '2.4',
       '2.5', '2.6', '3.1', '3.2', '3.3', '3.4', '3.5', '3.6', '4.1',
       '4.2', '4.3', '4.4', '4.5', '4.6', '4.7', '5.1', '5.2', '5.3',
       '5.4', '5.5', '5.6', '6.1', '6.2', '6.3', '6.4', '6.5', '6.6',
       '7.1', '7.2', '7.3', '7.4', '7.5', '7.6', '8.1', '8.2', '8.3',
       '8.4', '8.5', '8.6', '9.1', '9.2', '9.3', '9.4', '9.5', '9.6',
       '10.1', '10.2', '10.3', '10.4', '10.5', '10.6', '11.1', '11.2',
       '11.3', '11.4', '11.5', '11.6', '12.1', '12.2', '12.3', '12.4',
       '12.5', '12.6', '12.7', '13.1', '13.2', '13.3', '13.4', '13.5',
       '13.6', '14.1', '14.2', '14.3', '14.4', '14.5', '14.6', '15.1',
       '15.2', '15.3', '15.4', '15.5', '15.6', '15.7', '16.1', '16.2',
       '16.3', '16.4', '16.5', '16.6', '17.1', '17.2', '17.3', '17.4',
       '17.5', '17.6', '17.7', '18.1', '18.2', '18.3', '18.4', '18.5',
       '18.6'

In [54]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 178949 entries, 0 to 226
Data columns (total 40 columns):
batsman               178940 non-null object
batsman_runs          178940 non-null float64
batting_team          178949 non-null object
bowler                178940 non-null object
byes                  480 non-null float64
extras_runs           178299 non-null float64
fielders              6338 non-null object
kind                  8827 non-null object
legbyes               2933 non-null float64
inning                178949 non-null object
delivery              178949 non-null object
match_id              178949 non-null object
noballs               712 non-null float64
non_boundary          14 non-null float64
non_striker           178940 non-null object
player_out            8827 non-null object
total                 178940 non-null float64
wides                 5405 non-null float64
penalty               2 non-null float64
city                  175811 non-null object
competition  

In [0]:
df_ipl_summary.to_csv('ipl_summary.csv')

In [56]:
df_details.iloc[:,:].isna().mean()

batsman         0.000050
batsman_runs    0.000050
batting_team    0.000000
bowler          0.000050
byes            0.997318
extras_runs     0.003632
fielders        0.964582
kind            0.950673
legbyes         0.983610
inning          0.000000
delivery        0.000000
match_id        0.000000
noballs         0.996021
non_boundary    0.999922
non_striker     0.000050
player_out      0.950673
total           0.000050
wides           0.969796
penalty         0.999989
dtype: float64

In [0]:
df_combined.to_csv('ball_by_ball_details.csv')

In [58]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 178949 entries, 0 to 226
Data columns (total 40 columns):
batsman               178940 non-null object
batsman_runs          178940 non-null float64
batting_team          178949 non-null object
bowler                178940 non-null object
byes                  480 non-null float64
extras_runs           178299 non-null float64
fielders              6338 non-null object
kind                  8827 non-null object
legbyes               2933 non-null float64
inning                178949 non-null object
delivery              178949 non-null object
match_id              178949 non-null object
noballs               712 non-null float64
non_boundary          14 non-null float64
non_striker           178940 non-null object
player_out            8827 non-null object
total                 178940 non-null float64
wides                 5405 non-null float64
penalty               2 non-null float64
city                  175811 non-null object
competition  

In [59]:
df_combined.iloc[:,:].isna().mean()

batsman               0.000050
batsman_runs          0.000050
batting_team          0.000000
bowler                0.000050
byes                  0.997318
extras_runs           0.003632
fielders              0.964582
kind                  0.950673
legbyes               0.983610
inning                0.000000
delivery              0.000000
match_id              0.000000
noballs               0.996021
non_boundary          0.999922
non_striker           0.000050
player_out            0.950673
total                 0.000050
wides                 0.969796
penalty               0.999989
city                  0.017536
competition           0.000000
decision              0.000000
eliminator            0.987494
gender                0.000000
match_type            0.000000
method                0.982073
neutral_venue         0.897485
#_of_overs            0.000000
player_of_match       0.002079
result                0.985415
teams                 0.000000
umpires               0.001386
venue   

In [60]:
df_combined.result.value_counts()

tie          2238
no result     372
Name: result, dtype: int64

In [61]:
df_ipl_summary.result.value_counts()

tie          9
no result    4
Name: result, dtype: int64

In [62]:
df_details.info()

<class 'pandas.core.frame.DataFrame'>
Index: 178949 entries, 0 to 226
Data columns (total 19 columns):
batsman         178940 non-null object
batsman_runs    178940 non-null float64
batting_team    178949 non-null object
bowler          178940 non-null object
byes            480 non-null float64
extras_runs     178299 non-null float64
fielders        6338 non-null object
kind            8827 non-null object
legbyes         2933 non-null float64
inning          178949 non-null object
delivery        178949 non-null object
match_id        178949 non-null object
noballs         712 non-null float64
non_boundary    14 non-null float64
non_striker     178940 non-null object
player_out      8827 non-null object
total           178940 non-null float64
wides           5405 non-null float64
penalty         2 non-null float64
dtypes: float64(9), object(10)
memory usage: 27.3+ MB


In [63]:
df_details.non_boundary.notna().sum()

14

In [64]:
df_details[df_details.non_boundary.notna()]

,batsman,batsman_runs,batting_team,bowler,byes,extras_runs,fielders,kind,legbyes,inning,delivery,match_id,noballs,non_boundary,non_striker,player_out,total,wides,penalty
195,JEC Franklin,6.0,Mumbai Indians,B Kumar,NaN,0.0,NaN,NaN,NaN,inning_2,11.3,548308,NaN,1.0,SA Yadav,NaN,6.0,NaN,NaN
18,V Kohli,4.0,Royal Challengers Bangalore,DS Kulkarni,NaN,0.0,NaN,NaN,NaN,inning_1,3.1,980987,NaN,1.0,CH Gayle,NaN,4.0,NaN,NaN
167,V Kohli,6.0,Royal Challengers Bangalore,A Mishra,NaN,0.0,NaN,NaN,NaN,inning_2,7.1,548376,NaN,1.0,AB de Villiers,NaN,6.0,NaN,NaN
156,CH Gayle,4.0,Royal Challengers Bangalore,J Botha,NaN,0.0,NaN,NaN,NaN,inning_2,5.1,501252,NaN,1.0,TM Dilshan,NaN,4.0,NaN,NaN
186,YK Pathan,6.0,Kolkata Knight Riders,Yuvraj Singh,NaN,0.0,NaN,NaN,NaN,inning_2,10.3,598061,NaN,1.0,RN ten Doeschate,NaN,6.0,NaN,NaN
122,MM Sharma,4.0,Kings XI Punjab,TA Boult,NaN,0.0,NaN,NaN,NaN,inning_1,18.6,1082601,NaN,1.0,VR Aaron,NaN,4.0,NaN,NaN
190,RG Sharma,6.0,Mumbai Indians,Kuldeep Yadav,NaN,0.0,NaN,NaN,NaN,inning_2,9.7,980909,NaN,1.0,HH Pandya,NaN,6.0,NaN,NaN
123,LR Shukla,6.0,Kolkata Knight Riders,RP Singh,NaN,0.0,NaN,NaN,NaN,inning_1,19.6,336014,NaN,1.0,DJ Hussey,NaN,6.0,NaN,NaN
15,PC Valthaty,4.0,Kings XI Punjab,IK Pathan,NaN,0.0,NaN,NaN,NaN,inning_1,2.3,501257,NaN,1.0,AC Gilchrist,NaN,4.0,NaN,NaN
151,R Dravid,4.0,Rajasthan Royals,R Vinay Kumar,NaN,0.0,NaN,NaN,NaN,inning_2,4.4,501225,NaN,1.0,SR Watson,NaN,4.0,NaN,NaN
